In [ ]:
import s3fs
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt 
 

In [ ]:
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })

In [ ]:
fgs.ls('data')

In [ ]:
data_path = 's3://data/*.nc'

In [ ]:
remote_files = fsg.glob(data_path)
remote_files

In [ ]:
fileset = [fsg.open(file) for file in remote_files]
#combine all datasets on coords
dset = xr.open_mfdataset(fileset, combine='by_coords')

In [ ]:
# files needed 
v = xr.open_mfdataset(filedir + 'vbar.nc')
u = xr.open_mfdataset(filedir + 'ubar.nc')
bat = xr.open_mfdataset(filedir + '/CREG12.L75.script/bathym.nc')
dyp = xr.open_mfdataset(filedir + 'depth.nc')
cmap_r = plt.get_cmap("magma_r")

In [ ]:
# Velocity data 
u = u*73
v = v*73
vel = (u.vozocrtx**2 + v.vomecrty**2)**0.5

In [ ]:
# Smoothening Velocity data
us = u.rolling(x=10).mean()
vs = v.rolling(y=10).mean()
vels = (us.vozocrtx**2 + vs.vomecrty**2)**0.5


In [ ]:
# Quiver

c = 15
N = 100

x = np.linspace(0, 1801, 1801)
dx = 20 
x.shape

y = np.linspace(0, 1580, 1580)
dy = 20
y.shape

X, Y = np.meshgrid(x, y)
print(X.shape)
print(us.vozocrtx.shape)


In [ ]:
# chooseing a level to look at 
level = 15
depth = dyp.depth[level].values
print(depth)

y_grid = np.array(Y.T[800:1800:dx, 200:1400:dy])
print(y_grid.shape)
x_grid = np.array(X.T[800:1800:dx, 200:1400:dy])
u_grid = np.array(us.vozocrtx[level,800:1800:dx, 200:1400:dy].values)#endring
print(u_grid.shape)
v_grid = np.array(vs.vomecrty[level, 800:1800:dx, 200:1400:dy].values)
vel_grid = np.array(vels[level,800:1800:dx, 200:1400:dy].values)


In [ ]:
# plotting the quiver
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

ax.contour(bat.mbathy, colors='grey', levels=8, linewidths=0.4)

c = ax.quiver(y_grid, 
        x_grid,
        u_grid/vel_grid,
        v_grid/vel_grid,
        vel_grid,
        scale = 40,  
        width=0.003,
        #headwidth=2,
        #headlength=1,
        cmap = cmap_r) 

fig.colorbar(c,orientation='vertical', pad = 0.04, aspect=50, extendrect=True)
         
plt.title(f'Quiverplot of the current at '+ str(depth) + 'm depth')
# Arctic
ax.set_xlim(200,1400)
ax.set_ylim(800,1800)

plt.show()